In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.keras.callbacks import EarlyStopping


In [2]:

data = pd.read_csv('D:\sih_prototype\DWLR_Dataset_2023.csv')
features = ['Water_Level_m', 'Temperature_C', 'Rainfall_mm', 'pH', 'Dissolved_Oxygen_mg_L']

In [3]:
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length + 1):
        sequences.append(data[i:i+seq_length])
    return np.array(sequences)

seq_length = 3
X = create_sequences(scaled_data, seq_length)

NameError: name 'scaled_data' is not defined

In [ ]:
input_dim = X.shape[2]
latent_dim = 16

inputs = Input(shape=(seq_length, input_dim))
encoded = LSTM(latent_dim, activation='relu')(inputs)
decoded = RepeatVector(seq_length)(encoded)
decoded = LSTM(latent_dim, activation='relu', return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(input_dim))(decoded)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = autoencoder.fit(
    X, X,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    shuffle=True
)

In [ ]:
def detect_anomalies(model, data, threshold):
    reconstructions = model.predict(data)
    mse = np.mean(np.power(data - reconstructions, 2), axis=(1, 2))
    return mse > threshold

In [ ]:
threshold = np.mean(history.history['loss']) + 2 * np.std(history.history['loss'])
anomalies = detect_anomalies(autoencoder, X, threshold)

In [ ]:
anomaly_indices = np.where(anomalies)[0]
print("Anomalies detected at indices:", anomaly_indices)

In [ ]:
for idx in anomaly_indices:
    print(f"Anomaly at sequence {idx}:")
    print(data.iloc[idx:idx+seq_length][features])
    print("---")